# Semantic Kernal Chat Completion Agent - AzureChatCompletion Example

Travel Agent

## Install Dependencies

In [ ]:
%pip install semantic-kernel

## Create a Plugin
https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/function-calling/?pivots=programming-language-python

In [6]:
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import Annotated

# Define a sample plugin for the sample
class TravelPlugin:
    """A sample Travel Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of countries.")
    def get_countries(self) -> Annotated[str, "Returns the countries that you can book."]:
        return """
            United States
            Australia
            France
        """
    
    @kernel_function(description="Provides the hotels based on the country")
    def get_hotels(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the hotels for the country."]:
        if country == "United States":
            return """
                US Hotels: US Hotel 1, US Hotel 2, US Hotel 3, US Hotel 4, US Hotel 5
            """
        elif country == "Australia":
            return """
                Australia Hotels: Australia Hotel 6, Australia Hotel 7, Australia Hotel 8, Australia Hotel 9, Australia Hotel 10
            """
        elif country == "France":
            return """
                France Hotels: France Hotel 11, France Hotel 12, France Hotel 13, France Hotel 14, France Hotel 15            
            """
        else:
            return "Country not found"

    @kernel_function(description="Provides the typical travel budget for a country")
    def get_travel_budget(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the budget for the country."]:
        if country == "United States":
            return """
                Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "Australia":
            return """
                Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
                Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "France":
            return """
                Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.            
            """
        else:
            return "Country not found"
    
    @kernel_function(description="Provides the weather for a country")
    def get_weather(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the weather for the country."]:
        if country == "United States":
            return """
                The United States has a diverse climate. In general:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
                - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
        elif country == "Australia":
            return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
        elif country == "France":
            return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
        else:
            return "Country not found"
    

## Helper Method

With ChatCompletionAgent, developer must manually maintain the Chat History

In [2]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.kernel import Kernel


# A helper method to invoke the agent with the user input, manage chat history, and print the messages
async def invoke_agent(agent: ChatCompletionAgent, input: str, chat: ChatHistory) -> None:
    """Invoke the agent with the user input."""

    # Add the user message to the chat history
    chat.add_user_message(input)

    # Print the user input
    print(f"# {AuthorRole.USER}: '{input}'")
    
    async for content in agent.invoke(chat):
        # Print the agent response
        print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
        # Add the agent message to the chat history
        chat.add_message(content)
        await asyncio.sleep(5)


In [3]:
# Create the instance of the Kernel
kernel = Kernel()

azure_service_id = "azure"
kernel.add_service(AzureChatCompletion(service_id=azure_service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=azure_service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

kernel.add_plugin(TravelPlugin(), plugin_name="TravelPlugin")

# Create the agent
agent = ChatCompletionAgent(
    service_id=azure_service_id, 
    kernel=kernel, 
    name="TravelAgent", 
    instructions="Answer questions about the travel.", 
    execution_settings=settings
)

# Define the chat history
history = ChatHistory()

## Accepting User Input and Displaying Agent Response

Sample Questions:
1. What countries can I go to?
2. What is the weather in the United States?
3. If I have a budget of $300, where can I go for 4 days?

In [4]:
user_input = "What countries can I go to?"
await invoke_agent(agent, user_input, history)


# AuthorRole.USER: 'What countries can I go to?'
# AuthorRole.ASSISTANT - TravelAgent: 'You can travel to the following countries:
1. United States
2. Australia
3. France'


In [7]:
user_input = "What hotel can I stay in France?"
await invoke_agent(agent, user_input, history)


# AuthorRole.USER: 'What hotel can I stay in France?'
# AuthorRole.ASSISTANT - TravelAgent: 'In France, you can stay in the following hotels:
1. Hotel 11
2. Hotel 12
3. Hotel 13
4. Hotel 14
5. Hotel 15'


In [8]:
history.messages

[ChatMessageContent(inner_content=None, ai_model_id=None, metadata={}, content_type='message', role=<AuthorRole.USER: 'user'>, name=None, items=[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='What countries can I go to?', encoding=None)], encoding=None, finish_reason=None),
 ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BAUBuY0t3QE0lKfM6fq3NpeAvSxu9', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0x0Y96dK4vxPxHvMFsS8xe7n', function=Function(arguments='{}', name='TravelPlugin-get_countries'), type='function')]), content_filter_results={})], created=1741839034, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ded0d14823', usage=CompletionUsage(completion_tokens=14, prompt_tokens=152, total_tokens=166, comple

In [9]:
user_input = "What is the weather in the United States?"
await invoke_agent(agent, user_input, history)

# AuthorRole.USER: 'What is the weather in the United States?'
# AuthorRole.ASSISTANT - TravelAgent: 'The weather in the United States varies greatly across the country due to its diverse climate:

- **Winter**: 
  - Cold in the northern regions.
  - Mild in the southern regions.
  - Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.

- **Summer**: 
  - Hot and humid in the southern regions.
  - Dry in the western regions.
  - Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).'


In [10]:
user_input = "If I have a budget of $300, where can I go for 4 days?"
await invoke_agent(agent, user_input, history)

# AuthorRole.USER: 'If I have a budget of $300, where can I go for 4 days?'
# AuthorRole.ASSISTANT - TravelAgent: 'Based on a budget of $300 for 4 days, here's where you can go:

### United States
- **Budget Travelers**: $121 per day
  - Total Cost for 4 days: $484
  - Unfortunately, this exceeds your budget of $300.

### Australia
- **Budget Travelers**: $75 per day
  - Total Cost for 4 days: $300
  - You can afford a budget trip to Australia.

### France
- **Budget Travelers**: $100 per day
  - Total Cost for 4 days: $400
  - This exceeds your budget of $300. 

Therefore, you can afford a budget trip to Australia for 4 days with your $300 budget.'
